In [1]:
import pandas as pd
import matplotlib as mlp
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
trainingdf = pd.read_csv('./data/training_set_VU_DM.csv')

In [17]:
#Add column with day of the week
trainingdf['date_time'] = pd.to_datetime(trainingdf['date_time'])
trainingdf['weekday'] = trainingdf['date_time'].dt.dayofweek
trainingdf['weekday name'] = trainingdf['date_time'].dt.day_name()

#Add column with part of day: morning, afternoon, evening, night
trainingdf['part of day'] = pd.cut(trainingdf['date_time'].dt.hour,[0,6,12,18,24],labels=['night','morning','afternoon','evening'],include_lowest=True)

#Add column with season: winter, spring, summer, autumn
# Doesn't work since the mins must increase monotonically? Other option I say is 
# trainingdf['season'] = pd.cut(trainingdf['date_time'].dt.strftime('%m-%d'),[1-1, 3-21, 6-21, 21-9, 12-21, 12-31],labels=['winter','spring','summer','autumn', 'winter'],include_lowest=True)
trainingdf['season'] = pd.cut(trainingdf['date_time'].dt.month,[0,3,6,9,12],labels=['winter','spring','summer','autumn'],include_lowest=True)

trainingdf.head()


,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,date,weekday,weekday name,part of day,season
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,0.0,NaN,0,NaN,0,2013-04-04 08:32:15,3,Thursday,morning,spring
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,0.0,NaN,0,NaN,0,2013-04-04 08:32:15,3,Thursday,morning,spring
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,0.0,NaN,0,NaN,0,2013-04-04 08:32:15,3,Thursday,morning,spring
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,0.0,5.0,0,NaN,0,2013-04-04 08:32:15,3,Thursday,morning,spring
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,0.0,NaN,0,NaN,0,2013-04-04 08:32:15,3,Thursday,morning,spring
